In [1]:
from dataset_io import *
from classifiers import *
from tqdm import tqdm

In [2]:
#np.save("cross_validated_scenes",cross_validated_scenes())
cross_validated_scenes = np.load("cross_validated_scenes.npy", allow_pickle=True)

In [3]:
grad_metrics=[]
color_metrics=[]

for fold in tqdm(cross_validated_scenes):
    train_scenes, test_scenes, name = fold
    print(name)
    acc, prec, rec, f1, _ = classify_gradient(train_scenes, test_scenes, name, True)
    grad_metrics.append([acc, prec, rec, f1])
    res = classify_colors(train_scenes, test_scenes, name, True)
    color_metrics.append(res)
    
np.save("color_metrics", color_metrics)
np.save("grad_metrics", grad_metrics)

  0%|          | 0/4 [00:00<?, ?it/s]

fold_0
blue
1
2
3
31
32
4
5
6
7
8
9
10
dark
1
2
3
31
32
4
5
6
7
8
9
10
wooden
1
2
3
31
32
4
5
6
7
8


 25%|██▌       | 1/4 [03:54<11:42, 234.32s/it]

9
10
fold_1
blue
1
2
3
31
32
4
5
6
7
8
9
10
dark
1
2
3
31
32
4
5
6
7
8
9
10
wooden
1
2
3
31
32
4
5
6
7
8


 50%|█████     | 2/4 [08:16<08:05, 242.80s/it]

9
10
fold_2
blue
1
2
3
31
32
4
5
6
7
8
9
10
dark
1
2
3
31
32
4
5
6
7
8
9
10
wooden
1
2
3
31
32
4
5
6
7
8


 75%|███████▌  | 3/4 [12:14<04:01, 241.30s/it]

9
10
fold_3
blue
1
2
3
31
32
4
5
6
7
8
9
10
dark
1
2
3
31
32
4
5
6
7
8
9
10
wooden
1
2
3
31
32
4
5
6
7
8


100%|██████████| 4/4 [16:22<00:00, 245.52s/it]

9
10


In [11]:
# color_metrics = np.mean(color_metrics)
# grad_metrics = np.mean(grad_metrics)
#print(color_metrics)
print(grad_metrics, np.mean(grad_metrics, 0))

[[0.9978768577494692, 0.9978632478632479, 0.9915074309978769, 0.9946751863684771], [0.9939437121482009, 1.0, 0.9795918367346939, 0.9896907216494846], [0.934250107434465, 0.9957081545064378, 0.7544715447154472, 0.8584643848288621], [0.9970426700464723, 1.0, 0.9896602658788775, 0.9948032665181886]] [0.98077834 0.99839285 0.92880777 0.95940839]


In [5]:
-1. 4 features per all scenes available
-2. get folds
3. for each fold set it as test and load features for others (concat)
4. train models on loaded features
5. pass models to algorithm
6. run algorithm to predict single test fold
7. save predictions in list
8. mean of predctions

SyntaxError: invalid syntax (<ipython-input-5-675d1e1eb316>, line 1)